# **RAIN & Meta AI Academy: Building Agent Teams with LangGraph**

**Tutor:** Victor Olufemi
**Duration:** 2 Hours

### **Welcome to the Frontier of AI!**

Over the past weeks, we've gone on an incredible journey:
*   We **built** models with PyTorch.
*   We **deployed** models in apps with Streamlit.
*   We built a single **agent** that could use tools.
*   We gave an AI expert **knowledge** with RAG.

Today, we take the final and most exciting step: we will build a **team of AI agents that collaborate, critique, and improve their work** to solve a complex problem. We're moving from a single worker to an entire digital assembly line, powered by **LangGraph**.

### **Why LangGraph? The Problem with Simple Agent Chains**

Our previous agent was powerful but worked in a straight line. What if a task requires a loop? What if we need an agent to review work and send it *back* for revisions? This is where standard agent loops fall short.

LangGraph allows us to define agent workflows as a **state machine**.
*   **Analogy:** Think of it like a board game.
    *   **The State** is the game board, holding all the current information (the topic, search results, draft article).
    *   **The Nodes** are the players (our Searcher, Writer, and Reviewer agents). Each player takes a turn to change the board.
    *   **The Edges** are the rules that decide which player moves next, allowing for loops and complex decision-making.

Our goal is to build an agentic team that can write a well-researched article on any topic.


### **Step 0: Setup and Installation**

Let's get our Colab environment ready.


#### **1. Install Libraries**
LangGraph is a separate package from the core LangChain library.

In [ ]:
!pip install langgraph langchain langchain_groq duckduckgo-search python-dotenv langchain_community ddgs -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
# Groq Cloud API Key
import os
GROQ_API_KEY="gsk_.............................."
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
print("✅ Groq API Key loaded.")

✅ Groq API Key loaded.


### **Part 1: Defining the "State" - The Team's Shared Blackboard**

The `State` is a central object that all our agents can read from and write to. It's how they pass information to each other.

In [ ]:
from typing import List, TypedDict

class AgentState(TypedDict):
    """
    Represents the state of our agentic writing team.
    """
    topic: str
    search_results: List[str]
    draft: str
    review_critique: str
    revision_number: int
    final_article: str

This is the "memory" of our system. `topic` is the input, `search_results` are passed from Searcher to Writer, `draft` from Writer to Reviewer, and `review_critique` is the feedback loop. `revision_number` is a crucial safety mechanism to prevent infinite loops.

### **Part 2: Defining the Agents - The "Nodes"**

Each agent in our graph will be a simple Python function that takes the current state as input and returns a dictionary with the values it wants to update.

#### **1. The Searcher Agent**
This agent takes the topic and finds relevant information online.


In [ ]:
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

def search_agent(state: AgentState):
    """
    Node 1: The Searcher Agent.
    Takes a topic and searches for information online.
    """
    print("---🔍 SEARCHING ---")
    topic = state["topic"]
    search_query = f"about {topic}"
    search_results = search_tool.run(search_query)

    revision_number = state.get("revision_number", 0) + 1

    return {"search_results": search_results, "revision_number": revision_number}

#### **2. The Writer Agent**
This agent takes the search results and writes the first draft of the article.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

writer_llm = ChatGroq(model_name="meta-llama/llama-4-maverick-17b-128e-instruct")

writer_prompt = ChatPromptTemplate.from_template(
    """You are an expert article writer. Your task is to write a comprehensive, well-structured article
on the given topic, using ONLY the provided search results as context.

Do not use any of your own knowledge. The article should be at least 500 words long.

Topic: {topic}
Search Results:
{search_results}

Article Draft:
"""
)

def writer_agent(state: AgentState):
    """
    Node 2: The Writer Agent.
    Takes the search results and writes a draft article.
    """
    print("---✍️ WRITING DRAFT ---")
    topic = state["topic"]
    search_results = state["search_results"]

    # We create a simple chain for the writer
    writer_chain = writer_prompt | writer_llm | StrOutputParser()
    draft = writer_chain.invoke({"topic": topic, "search_results": search_results})

    revision_number = state.get("revision_number", 0) + 1 # Increment revision number

    return {"draft": draft, "revision_number": revision_number}

#### **3. The Reviewer Agent**
This is the critical agent. It reviews the draft and decides if it's good enough or needs revision.

In [ ]:
reviewer_llm = ChatGroq(model_name="meta-llama/llama-4-maverick-17b-128e-instruct")
 # A smaller model is fine for reviewing

# --- NEW, MORE POWERFUL PROMPT ---
reviewer_prompt = ChatPromptTemplate.from_template(
  """You are a pragmatic editor. Your goal is to ensure an article is factually correct, coherent, and reasonably comprehensive. Do not aim for perfection.

  Review the draft based on the topic. Your decision MUST be one of three choices:
  'DECISION: APPROVE'
  'DECISION: REVISE_WRITER'
  'DECISION: REVISE_SEARCHER'

  Follow these rules:
  1.  If the article is well-written and covers the main points of the topic, **APPROVE** it. It doesn't need to be perfect.
  2.  If the article has good information but is poorly structured or unclear, request a rewrite by choosing **REVISE_WRITER**.
  3.  If the article fundamentally lacks key information or seems factually thin, you MUST choose **REVISE_SEARCHER** to get better source material.

  Provide a concise reason for your decision, then end with the required DECISION line.

  Topic: {topic}
  Draft Article:
  {draft}

  Your Critique:
  """
)

def reviewer_agent(state: AgentState):
    """
    Node 3: The Reviewer Agent.
    Reviews the draft and decides to approve, request a rewrite, or request more research.
    """
    print("---🧐 REVIEWING DRAFT ---")
    topic = state["topic"]
    draft = state["draft"]

    # We pass BOTH topic and draft for better context
    reviewer_chain = reviewer_prompt | reviewer_llm | StrOutputParser()
    review_critique = reviewer_chain.invoke({"topic": topic, "draft": draft})

    print(f"Reviewer's Critique:\n{review_critique}")

    # This is a robust way to check the decision
    if "APPROVE" in review_critique:
        print("---✅ DRAFT APPROVED ---")
        return {"final_article": draft, "review_critique": review_critique}
    elif "REVISE_SEARCHER" in review_critique:
        print("---❌ DRAFT REJECTED: MORE RESEARCH NEEDED ---")
        return {"review_critique": review_critique}
    else: # Default to revising the writer if no other decision is clear
        print("---❌ DRAFT NEEDS REVISION (WRITER) ---")
        return {"review_critique": review_critique}

Go through each agent. Emphasize how each one is a self-contained unit of work that operates on the shared `State`. Highlight the prompts, as they are the "instructions" for each agent's brain.


### **Part 3: Defining the Logic - The "Edges"**

Now we define the rules of our board game. The most important rule is the conditional edge that creates our revision loop.


In [ ]:
def should_continue(state: AgentState):
    """
    This is our conditional edge. It decides where to go after the review.
    """
    print("---DECIDING NEXT STEP---")
    critique = state.get("review_critique", "")

    if "APPROVE" in critique:
        return "end"
    elif state["revision_number"] > 5:
        print("---MAX REVISIONS REACHED---")
        return "end"
    elif "REVISE_SEARCHER" in critique:
        print("---ROUTING TO SEARCHER---")
        return "research"
    else:
        print("---ROUTING TO WRITER---")
        return "revise"

This function is the heart of LangGraph's power. It's simple Python logic that directs the flow of the graph. Explain the importance of the `revision_number` check as a safety valve.


### **Part 4: Assembling the Graph**

Let's wire everything together.

In [ ]:
from langgraph.graph import StateGraph, END

# Initialize the graph
workflow = StateGraph(AgentState)

# Add the nodes
workflow.add_node("search_agent", search_agent)
workflow.add_node("writer_agent", writer_agent)
workflow.add_node("reviewer_agent", reviewer_agent)

# Define the edges
workflow.set_entry_point("search_agent")
workflow.add_edge("search_agent", "writer_agent")
workflow.add_edge("writer_agent", "reviewer_agent")

# Add the NEW conditional edge
workflow.add_conditional_edges(
    "reviewer_agent",
    should_continue,
    {
        "end": END,
        "revise": "writer_agent",  # If needs revision, go back to the writer
        "research": "search_agent", # If needs more info, go back to the searcher!
    },
)

In [ ]:
# 5. Compile the graph into a runnable app
app = workflow.compile()
print("✅ Advanced Agentic Writing Team Graph Compiled!")

✅ Advanced Agentic Writing Team Graph Compiled!


### **Part 5: Run the Agentic Writing Team!**

Now, let's give our team a topic and watch them collaborate. We'll use `.stream()` to see the step-by-step process.

In [ ]:
# The topic for our article
topic = "Victor Olufemi - Top AI professional in Africa"

In [ ]:
# Run the stream and print intermediate steps
for s in app.stream({"topic": topic}):
    print("---")
    node_that_just_ran = list(s.keys())[0]
    print(f"Node: '{node_that_just_ran}'")

    # Save the current state
    last_state = s

    state_after_node = s[node_that_just_ran]
    for key, value in state_after_node.items():
        if value is not None and value != '' and key != 'topic':
            print(f"- {key}: {str(value)[:300]}...")

---🔍 SEARCHING ---
---
Node: 'search_agent'
- search_results: Currently, my career is dedicated to technology for good, building ethical A.I tools that make a positive impact on people and the economy. I also participate in hackathons on Zindi Africa … Victor is optimistic about Africa ’s AI growth but acknowledges gaps. Victor Olufemi ’s journey into artifici...
- revision_number: 1...
---✍️ WRITING DRAFT ---
---
Node: 'writer_agent'
- draft: Victor Olufemi: Pioneering AI for Good in Africa

In the rapidly evolving landscape of artificial intelligence (AI), Africa is steadily making its mark, thanks to trailblazers like Victor Olufemi. A top AI professional in Africa, Olufemi has been making waves in the industry with his innovative appr...
- revision_number: 2...
---🧐 REVIEWING DRAFT ---
Reviewer's Critique:
The article on Victor Olufemi provides a comprehensive overview of his achievements and contributions to the field of AI in Africa. It highlights his background, notable achievem

In [ ]:
# After the loop finishes, last_state contains the final state of the graph
final_article = "No final article was produced."
if last_state:
    # The final state is nested under the last node that ran (usually the reviewer)
    last_node_name = list(last_state.keys())[0]
    final_article = last_state[last_node_name].get("final_article", "Article not approved or an error occurred.")


print("\n" + "="*50)
print("--- ARTICLE ---")
print("="*50)
print(final_article)


--- ARTICLE ---
Victor Olufemi: A Pioneer in AI for Good in Africa

In the rapidly evolving landscape of Artificial Intelligence (AI) and machine learning, few individuals have made a mark as significant as Victor Olufemi. As a top AI professional in Africa, Olufemi has been at the forefront of harnessing the power of technology to drive positive change across the continent. With a career dedicated to "technology for good," Olufemi has been instrumental in building ethical AI tools that have a profound impact on people and the economy.

Olufemi's journey into the world of AI and data science is a testament to his relentless curiosity and resilience. With a background in Electronic and Electrical Engineering from Nigeria, he has evolved into an award-winning data scientist, NLP researcher, and AI researcher. His expertise in machine learning and AI ethics has enabled him to apply data-driven approaches to solve real-world problems across various domains.

One of Olufemi's most notable 